In [2]:
import os
import preprocessing

from gensim import corpora

In [3]:
#texts = preprocessing.corpus_tokens()
#dictionary = corpora.Dictionary(texts)
flat_metadata = preprocessing.flat_unique()
summaries = [paper['summary'].replace('\n', ' ') for paper in flat_metadata]

# Sample
summaries[0][:200]

'Time series forecasting is a crucial component of many important applications, ranging from forecasting the stock markets to energy load prediction. The high-dimensionality, velocity and variety of th'

In [38]:
import spacy
from spacy.lang.en.examples import sentences

nlp = spacy.load('en_core_web_md')

In [39]:
from tqdm import tqdm

corpus_orgs = []
for i in tqdm(range(len(summaries))):
    doc = nlp(summaries[i])
    doc_orgs = []
    for entity in doc.ents:
        #print(entity.text, entity.label_)
        if entity.label_ == 'ORG':
            text = entity.text
            #print(text)
            doc_orgs.append(text)
    corpus_orgs.append(doc_orgs)

100%|██████████| 9232/9232 [07:04<00:00, 25.26it/s]


In [42]:
import time
import networkx as nx
from networkx.drawing.nx_pylab import draw_networkx, draw
import matplotlib.pyplot as plt

def gograph():
    G = nx.Graph()
    #G.add_edge('A', 'B', weight=4)
    #nx.shortest_path(G, 'A', 'B', weight='weight')
    sim = []
    N = 50
    #for i in range(1, len(summaries)):
    for i in tqdm(range(1, N)):
        for j in range(1, N):
            # Determine semantic similarities
            doc1 = nlp(summaries[i])
            doc2 = nlp(summaries[j])
            similarity = doc1.similarity(doc2)

            G.add_edge(str(j), str(i), weight=1-similarity)

    try:
        draw_networkx(G)
    except:
        pass
    finally:
        plt.draw()
    
    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(12, 5))
    plt.xlim(0.87, 1)
    sns.distplot(sim)
    plt.show()

In [1]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("glove-wiki-gigaword-300")  # download the model and return as object ready for use

In [29]:
import preprocessing
import gensim


def read_corpus(metadata_path='data/arxiv_metadata.json', tokens_only=False):
    arx = preprocessing.flat_unique(path=metadata_path)
    for i, paper in enumerate(arx):
            if tokens_only:
                yield gensim.utils.simple_preprocess(paper['title'] + paper['summary'])
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(paper['title'] + paper['summary']), [i])
    
    
#test = list(read_corpus(tokens_only=True))
train_corpus = list(read_corpus(tokens_only=False))

In [39]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=100)
model.build_vocab(train_corpus) # ?

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 9min 21s, sys: 5.46 s, total: 9min 27s
Wall time: 3min 21s


In [41]:
import random

# Get summaries
train_summaries = [paper['summary'] for paper in preprocessing.flat_unique()]
train_titles = [paper['title'] for paper in preprocessing.flat_unique()]

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_summaries) - 1)
inferred_vector = model.infer_vector(train_summaries[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(train_titles[doc_id], train_summaries[doc_id]))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], train_summaries[sims[index][0]]))

Test Document (On possible origins of trends in financial market price changes): «We investigate possible origins of trends using a deterministic threshold
model, where we refer to long-term variabilities of price changes (price
movements) in financial markets as trends. From the investigation we find two
phenomena. One is that the trend of monotonic increase and decrease can be
generated by dealers' minuscule change in mood, which corresponds to the
possible fundamentals. The other is that the emergence of trends is all but
inevitable in the realistic situation because of the fact that dealers cannot
always obtain accurate information about deals, even if there is no influence
from fundamentals and technical analyses.»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d300,n5,w5,mc2,s0.001,t3):

MOST (2048, 0.20832426846027374): «We present a solution for the problems related to the application of
multivariate Garch models to markets with a large number of stocks by
restricting the form